In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import interact_manual, Layout
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 70em; }</style>"))

directory = os.getcwd()
sys.path.insert(1, directory)

from Pesca.main import ModeloPesca
from Ganaderia.main import ModeloGanaderia
from Mineria.main import ModeloMineria

from Capitales.kfitt import main as mainkfitt
from Capitales.kft import main as mainkft
from Capitales.kht import main as mainkht
from Capitales.kst import main as mainkst


In [12]:
# SECONDARY FUNCTIONS
def input_as_decimal(des, default):
    val = widgets.FloatText(default, description=des, 
                            disabled=False,style={'description_width': 'initial'},
                            layout=Layout(width='50%'))
    return val

def input_as_list(des, default, list_selec):
    val = widgets.Dropdown(options=list_selec,
                           value=default,
                           description=des,
                           disabled=False,
                           style={'description_width': 'initial'},
                           layout=Layout(width='50%'))
    return val

<h1><center>Servicios ecosistémicos y medios de vida (SE/MV)</center></h1>

In [13]:
# Auxiliar variables
#                        Mes   Area ano neutro
NeuData = pd.DataFrame({('01', 36866.5),
                        ('02', 33807.5),
                        ('03', 34462.9),
                        ('04', 36866.5),
                        ('05', 44606.0),
                        ('06', 42230.3),
                        ('07', 38193.4),
                        ('08', 37909.5), 
                        ('09', 41032.1),
                        ('10', 42230.3),
                        ('11', 42230.3),
                        ('12', 42230.3)}, columns=['Mes', 'Area Neutro'])
        
#                      Mes   % De abundancia
PAbun = pd.DataFrame({('01', 0.05),
                      ('02', 0.05),
                      ('03', 0.05),
                      ('04', 0.30),
                      ('05', 0.30),
                      ('06', 0.05),
                      ('07', 0.05),
                      ('08', 0.05), 
                      ('09', 0.05),
                      ('10', 0.05),
                      ('11', 0.05),
                      ('12', 0.05)}, columns=['Mes', 'Porcentaje'])

cond_ini = False
cond_ini_pes = False
cond_ini_gan = False
cond_ini_min = False
cond_ini_merge = False

cond_ini_kst = False
cond_ini_kht = False
cond_ini_kfitt = False

<b>Trayectoria</b>
<p>Seleciona la trayectoria a trabajar.</p>

In [14]:
# SELECTOR DATA
names =[file[:-15].replace('_', ' ') for file in os.listdir('input')]
files = os.listdir('input')
@interact_manual(file = [(ii[0], ii[1]) for ii in list(zip(names, files))])
def selec_trayectoria(file):
    loc = os.path.join('input', file)
    df_init = pd.read_csv(loc)
    sitios = list(np.unique(df_init.loc[0]))
    sitios.remove('Sector')
    especies = list(np.unique(df_init.loc[1]))
    especies.remove('Especie')
    especies.remove('Nivel')
    
    @interact_manual(sitio = sitios,
                     especie = especies)
    def selec_sitio_especie(sitio, especie):
        
        global df_res, cond_ini
        
        col1 = list(np.array(df_init.columns[df_init.iloc[1] == especie]))
        col1 += list(np.array(df_init.columns[df_init.iloc[1] == 'Nivel']))
        col2 = list(np.array(df_init.columns[df_init.iloc[0] == sitio]))
        col3 = list(set(col1).intersection(col2))
        col3.insert(0, 'Fecha')
        
        df_slice = df_init[col3].copy()
        
        df_res = pd.DataFrame()
        df_res['Date']  = pd.to_datetime(df_slice.loc[3:, 'Fecha'], format='%Y-%m-%d').dt.strftime('%d-%m-%Y')
        df_res['WUA']   = df_slice.loc[3:, df_slice.columns[df_slice.iloc[2] == 'WUA ha']]
        df_res['Area']  = df_slice.loc[3:, df_slice.columns[df_slice.iloc[2] == 'Area ha']]
        df_res['Nivel'] = df_slice.loc[3:, df_slice.columns[df_slice.iloc[2] == 'Promedio_Nivel_(m)']]
        df_res.reset_index(drop=True, inplace=True)
        
        df_res['WUA'] = df_res['WUA'].astype(float)
        df_res['Area'] = df_res['Area'].astype(float)
        df_res['Nivel'] = df_res['Nivel'].astype(float)
        
        print('Trayectoria selecionada correctamente.')
        cond_ini = True

interactive(children=(Dropdown(description='file', options=(('Año Tipo Humedo ', 'Año_Tipo_Humedo_SocialInput.…

<center><b>Modelo de Servicios ecosistémicos y de medios de vida</b></center>
<b>Modelo de Pesca</b>

<div>
<p>Ecuaciones generales del modelo:</p>
<br>
<center>
$\frac{Abundancia}{dt} = Reclutamiento - Decrecimiento - Captura$
<br>
<br>
$\frac{Capital financiero}{dt} = Ventas - Costos$
</center>
<br>
</div>

Los valores presentados a continuacion por default, se refieren a la especie Bagre Rayado. En caso de trabajar una especie diferente, es necesario cambiar estos valores.

In [23]:
#@interact_manual()
def mod_pes_run(v1, v2, v3, v4, v5, v6, v7, v8, v9, v10,
                v11, v12, v13, v14, v15, v16, v17, v18, v19,
                v20, v21):
    if cond_ini == True:
        # Modelo pesca
        global prereclutas_res, abundacia_res, capital_financiero_res, cond_ini_pes, clima_tipo
        
        # Parámetros
#         par_prereclutas = {'Individuos por ha' : 4,
#                            'Porcentaje hembras': 0.5,
#                            'Porcentaje maduros': 0.5,
#                            'Desove por individuo': 981000,
#                            'Porcentaje de abundancia': PAbun,
#                            'Porcentaje de supervivencia': 0.001/100}
#         par_serv_ecosi = {'Tasa de mortalidad' : 0.0383333, 
#                           'Pescadores': 1323,
#                           'Captura potencial promedio': 10.5,
#                           'Porcentaje de captura': 0.2186,
#                           'Peso por individuo - promedio' : 4.58,
#                           'Peso por individuo - des. estandar' : 3.60,
#                           'Peso por individuo - mínimo': 0.216,
#                           'Peso por individuo - máximo': 27.460}
#         par_cap_fin = {'Porcentaje de autoconsumo' : 0.013,
#                        'Precio por libra - promedio' : 4388.631217,
#                        'Precio por libra - minimo' : 700,
#                        'Precio por libra - maximo' : 10000,
#                        'Precio por libra - desv. estandar' : 1076.245027,
#                        'Costos mensuales': 18943*30}

        par_prereclutas = {'Individuos por ha' : v1,
                           'Porcentaje hembras': v2,
                           'Porcentaje maduros': v3,
                           'Desove por individuo': v4,
                           'Porcentaje de abundancia': PAbun,
                           'Porcentaje de supervivencia': v5}
        par_serv_ecosi = {'Tasa de mortalidad' : v6, 
                          'Pescadores': v7,
                          'Captura potencial promedio': v8,
                          'Porcentaje de captura': v9,
                          'Peso por individuo - promedio' : v10,
                          'Peso por individuo - des. estandar' : v11,
                          'Peso por individuo - mínimo': v12,
                          'Peso por individuo - máximo': v13}
        par_cap_fin = {'Porcentaje de autoconsumo' : v14,
                       'Precio por libra - promedio' : v15,
                       'Precio por libra - minimo' : v16,
                       'Precio por libra - maximo' : v17,
                       'Precio por libra - desv. estandar' : v18,
                       'Costos mensuales': v19}
        
        # Valores iniciales
        abundancia_ini = int(par_prereclutas['Individuos por ha'] * df_res.loc[0, 'Area']) #107013
        cap_financiero_init = 0
        
        foo = ModeloPesca(df_res, NeuData)
        foo.dictionary['vulnerabilidad humedo'] = v20
        foo.dictionary['vulnerabilidad seco'] = v21
        
        prereclutas_res = foo.prereclutas_fun(par_prereclutas)
        abundacia_res = foo.servicio_ecosistemico_fun(abundancia_ini, prereclutas_res, par_serv_ecosi)
        capital_financiero_res = foo.capital_financiero_fun(cap_financiero_init, abundacia_res, par_cap_fin)
        print('Modelo de pesca ejecutado.')
        
        clima_tipo = list(foo.wua_df['type'])
        
        cond_ini_pes = True
    else:
        print('Por favor, selecione la trayectoria.')

print('')
print('Parámetros del modelo de pesca: ')
print('')
w = interact_manual(mod_pes_run,
                    v1 = input_as_decimal('Individuos por ha', 4),
                    v2 = input_as_decimal('Porcentaje hembras', 0.5),
                    v3 = input_as_decimal('Porcentaje maduros', 0.5),
                    v4 = input_as_decimal('Desove por individuo', 981000),
                    v5 = input_as_decimal('Porcentaje de supervivencia', 0.001),
                    
                    v6 = input_as_decimal('Tasa de mortalidad', 0.0383333),
                    v7 = input_as_decimal('Pescadores', 1323),
                    v8 = input_as_decimal('Captura potencial promedio', 10.5),
                    v9 = input_as_decimal('Porcentaje de captura', 0.2186),
                    v10 = input_as_decimal('Peso por individuo - promedio', 4.58),
                    v11 = input_as_decimal('Peso por individuo - des. estandar', 3.60),
                    v12 = input_as_decimal('Peso por individuo - mínimo', 0.216),
                    v13 = input_as_decimal('Peso por individuo - máximo', 27.460),
                    
                    v14 = input_as_decimal('Porcentaje de autoconsumo', 0.013),
                    v15 = input_as_decimal('Precio por libra - promedio', 4388.631217),
                    v16 = input_as_decimal('Precio por libra - minimo', 700),
                    v17 = input_as_decimal('Precio por libra - maximo', 10000),
                    v18 = input_as_decimal('Precio por libra - desv. estandar', 1076.245027),
                    v19 = input_as_decimal('Costos mensuales', 18943*30),
                    
                    v20 = input_as_decimal('Vulnerabilidad ante periodos secos', 0.5),
                    v21 = input_as_decimal('Vulnerabilidad ante periodos humedos', 0.216))


Parámetros del modelo de pesca: 



interactive(children=(FloatText(value=4.0, description='Individuos por ha', layout=Layout(width='50%'), style=…

<b>Modelo de ganadería</b>

<p>Ecuaciones generales del modelo:</p>
<br>
<center>
$\frac{Área\_de\_pastos}{dt} = Deforestación + Potrerización + Cultivos\_a\_pastos + Desecación - Reforestación - Pastos\_a\_cultivos - Inudación$
<br>
<br>
<center>
$\frac{Cabezas\_de\_ganado\_por\_ha}{dt} = Cabezas\_nuevas - Cabezas\_vendidas - Muertes\_de\_cabezas\_de\_ganado$
<br>
<br>
$\frac{Capital\_financiero}{dt} = Ventas - Inversión - Disminución\_de\_KF$
</center>
<br>
    


In [24]:
#def mod_pes_gan():
def mod_pes_gan(v1, v2, v3, v4, v5, v6, v7, v8, v9, v10,
                v11, v12, v13, v14):
    if cond_ini == True:
        # Modelo de ganadería
        global eco_df, gan_df, kfg_df, cond_ini_gan
        
        para = {#'Tasa de reforestación' : 0.00116667,
                #'Tasa de deforestación' : 0.12,
                #'Tasa de pastos a cultivos' : 0.24,
                #'Tasa de cultivos a pastos' : 0.24,
                #'Tasa de poterización' : 0.02,
                #'Tiempo de desinundación' : 1,

                #'CC por hectareas' : 2,
                #'Tasa de mortalidad natural' : 0.1,
                #'Porcentaje por ventas' : 0.2,
                
                'Tasa de reforestación' : v1,
                'Tasa de deforestación' : v2,
                'Tasa de pastos a cultivos' : v3,
                'Tasa de cultivos a pastos' : v4,
                'Tasa de poterización' : v5,
                'Tiempo de desinundación' : v6,

                'CC por hectareas' : v7,
                'Tasa de mortalidad natural' : v8,
                'Porcentaje por ventas' : v9,
                
                'Tasa de natalidad vs rel. vacas - Tasa de natalidad': [0.0,
                                                                        0.2,
                                                                        0.4,
                                                                        0.6,
                                                                        0.8,
                                                                        1.0,
                                                                        1.2,
                                                                        1.4,
                                                                        1.6,
                                                                        1.8,
                                                                        2.0],
                'Tasa de natalidad vs rel. vacas - Relación vacas CC': [0.9933,
                                                                        0.982,
                                                                        0.9526,
                                                                        0.8808,
                                                                        0.7311,
                                                                        0.3,
                                                                        0.192,
                                                                        0.1192,
                                                                        0.04743,
                                                                        0.01799,
                                                                        0.006693],

#                 'Precio por macho' : 2090000,
#                 'Precio por hembra' : 1382500,
#                 'Fración machos' : 0.38,
#                 'Fracción hembras' : 0.62,
#                 'Insumos' : 0.9,
            
                'Precio por macho' : v10,
                'Precio por hembra' : v11,
                'Fración machos' : v12,
                'Fracción hembras' : v13,
                'Insumos' : v14,
                
                'Afectación al KF vs Pas. inundados - Pas. inundados' : [0.0000,
                                                                         0.09091,
                                                                         0.1818,
                                                                         0.2727,
                                                                         0.3636,
                                                                         0.4545,
                                                                         0.5455,
                                                                         0.6364,
                                                                         0.7273,
                                                                         0.8182,
                                                                         0.9091,
                                                                         1.000],

                'Afectación al KF vs Pas. inundados - Afectación al KF' : [0.000,
                                                                           0.1818,
                                                                           0.3636,
                                                                           0.5455,
                                                                           0.7273,
                                                                           0.9091,
                                                                           1.091,
                                                                           1.273,
                                                                           1.455,
                                                                           1.636,
                                                                           1.818,
                                                                           2.000]
                }
        
        # Valores iniciales
        init_area = [549555, 138368, 289196, 242597, 0]
        init_gan = para['CC por hectareas']
        init_kfg = 0
        
        foo = ModeloGanaderia(hd_data=df_res, para=para)
        foo.precalc()
        eco_df = foo.eco_area(init_area)
        gan_df = foo.ganaderia(init_gan, eco_df)
        kfg_df = foo.kf_modelo_ganaderia(init_kfg, gan_df, eco_df)
        
        print('Modelo de ganaderia ejecutado.')
        cond_ini_gan = True
    else:
        print('Por favor, selecione la trayectoria.')
        
print('')
print('Parámetros del modelo de Ganadería: ')
print('')

w = interact_manual(mod_pes_gan, 
                    v1 = input_as_decimal('Tasa de reforestación', 0.00116667), 
                    v2 = input_as_decimal('Tasa de deforestación', 0.12), 
                    v3 = input_as_decimal('Tasa de pastos a cultivos', 0.24), 
                    v4 = input_as_decimal('Tasa de cultivos a pastos', 0.24), 
                    v5 = input_as_decimal('Tasa de poterización', 0.02), 
                    v6 = input_as_decimal('Tiempo de desinundación', 1), 
                    
                    v7 = input_as_decimal('CC por hectareas', 2), 
                    v8 = input_as_decimal('Tasa de mortalidad natural', 0.1), 
                    v9 = input_as_decimal('Porcentaje por ventas', 0.2), 
                    v10= input_as_decimal('Precio por macho', 2090000),
                    v11= input_as_decimal('Precio por hembra', 1382500), 
                    v12= input_as_decimal('Fración machos', 0.38), 
                    v13= input_as_decimal('Fracción hembras', 0.62), 
                    v14= input_as_decimal('Insumos', 0.9))


Parámetros del modelo de Ganadería: 



interactive(children=(FloatText(value=0.00116667, description='Tasa de reforestación', layout=Layout(width='50…

<b>Modelo de minería</b>

<p>Ecuaciones generales del modelo:</p>
<br>
<center>
$\frac{Oro\_extraido}{dt} = Extración\_de\_oro - Ventas$
<br>
<br>
<center>
$\frac{KF\_Minería}{dt} = Ingresos\_totales\_KFM - Costos\_minería$
<br>

In [25]:
def mod_pes_gan(v1, v2, v3, v4, v5):
    if cond_ini == True:
        # Modelo minería
        global oro_df, kfmin_df, cond_ini_min
        
        # Valores iniciales
        init_oro = 0
        init_kfm = 0
        
#         para = {'Nivel referencia de productividad': 28.5,
#                 'Productividad por área': 842.61355937,
#                 'Oro vendido': 302.6224991,
#                 'Precio gramo de oro' : 207842.96,
#                 'Costos': 95681.64}
        
        para = {'Nivel referencia de productividad': v1,
                'Productividad por área': v2,
                'Oro vendido': v3,
                'Precio gramo de oro' : v4,
                'Costos': v5}
        
        foo = ModeloMineria(hd_data=df_res, para=para)
        foo.precalc()
        oro_df = foo.oro_extraido(init_oro)
        kfmin_df = foo.kf_modelo_mineria(init_kfm, oro_df)
        
        print('Modelo de minería ejecutado.')
        cond_ini_min = True
    else:
        print('Por favor, minería la trayectoria.')
        
print('')
print('Parámetros del modelo de minería: ')
print('')

w = interact_manual(mod_pes_gan, 
                    v1 = input_as_decimal('Nivel referencia de productividad', 28.5),
                    v2 = input_as_decimal('Productividad por área', 842.61355937),
                    v3 = input_as_decimal('Oro vendido', 302.6224991),
                    v4 = input_as_decimal('Precio gramo de oro', 207842.96),
                    v5 = input_as_decimal('Costos', 95681.64))


Parámetros del modelo de minería: 



interactive(children=(FloatText(value=28.5, description='Nivel referencia de productividad', layout=Layout(wid…

<p>Resultados de los modelos</p>

In [26]:
def add_prefix(df, columns_to_prefix, prefix):
    res_tmp = pd.DataFrame()
    for column in columns_to_prefix:
        if column != 'Fecha':
            res_tmp[prefix + ' - ' + column] = df[column]
        else:
            res_tmp[column] = df[column]
    return res_tmp

@interact_manual()
def print_results():
    if cond_ini_pes and cond_ini_gan and cond_ini_min:
        global res_merged, cond_ini_merge, res_merged_fixed
        
        # Fix
        try:
            prereclutas_res['Fecha'] = prereclutas_res['Date']
            prereclutas_res.drop('Date', axis=1, inplace=True)

            abundacia_res['Fecha'] = abundacia_res['Date']
            abundacia_res.drop('Date', axis=1, inplace=True)

            capital_financiero_res['Fecha'] = capital_financiero_res['Date']
            capital_financiero_res.drop('Date', axis=1, inplace=True)
        except:
            print('Date column removed.')

#         eco_df['Fecha'] = eco_df['Date']
#         eco_df.drop('Date', axis=1, inplace=True)
        
        # Merge
        res = prereclutas_res.copy()
        res = pd.merge(res, abundacia_res, on='Fecha', how='left')
        res = pd.merge(res, capital_financiero_res, on='Fecha', how='left')
        
        res = pd.merge(res, eco_df, on='Fecha', how='left')
        res = pd.merge(res, gan_df, on='Fecha', how='left')
        res = pd.merge(res, kfg_df, on='Fecha', how='left')
        
        res = pd.merge(res, oro_df, on='Fecha', how='left')
        res = pd.merge(res, kfmin_df, on='Fecha', how='left')
        
        for name_col in res.columns:
            res.loc[res[name_col].isna(), name_col] = 0
        
        res_merged = res.copy()
        
        # MERGED FIXED
        res_p = prereclutas_res.copy()
        res_p = pd.merge(res_p, abundacia_res, on='Fecha', how='left')
        res_p = pd.merge(res_p, capital_financiero_res, on='Fecha', how='left')
        
        res_g = eco_df.copy()
        res_g = pd.merge(res_g, gan_df, on='Fecha', how='left')
        res_g = pd.merge(res_g, kfg_df, on='Fecha', how='left')
        
        res_m = oro_df.copy()
        res_m = pd.merge(res_m, kfmin_df, on='Fecha', how='left')
        
        res_p = add_prefix(res_p, res_p.columns, 'M. Pesca')
        res_g = add_prefix(res_g, res_g.columns, 'M. Ganaderia')
        res_m = add_prefix(res_m, res_m.columns, 'M. Mineria')
        
        res_merged_fixed = res_p.copy()
        res_merged_fixed = pd.merge(res_merged_fixed, res_g, on='Fecha', how='left')
        res_merged_fixed = pd.merge(res_merged_fixed, res_m, on='Fecha', how='left')
        
        for name_col in res_merged_fixed.columns:
            res_merged_fixed.loc[res_merged_fixed[name_col].isna(), name_col] = 0
        
        res_merged_fixed.index = res_p['Fecha']
        
        display(HTML(res_merged_fixed.to_html()))
        cond_ini_merge = True
        print('Copy to clipboard.')
        @interact_manual()
        def copy():
            res_merged_fixed.to_clipboard(index=True)
        
    else:
        print('Se requiere ejecutar todos los modelos')

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<h2><center>Medios de vida<br>(Capital financiero, físico, humano y social)</center></h2>

<p><b>Capital Social</b></p>

![kst](Capitales/kst.png)

In [27]:
def capSocial(v1, v2, v3, v4, v5, v6, v7, v8, v9, v10):
    if cond_ini_merge == True:
        
        global cond_ini_kst, kst_df
        
        init_val = 0
        para = {'Tasa de vinculación comunitaria':v1,
                'Tasa de grupo de apoyo ante eventos':v2,
                'Tasa de percepción positiva de ayudas del gobierno':v3,
                'Tasa de vinculación productiva':v4,
                'Tasa de no vinculación con organizaciones relacionadas a eventos externos':v5,
                'Tasa de vinculación familiar':v6,
                'Union de la comunidad ante eventos':v7,
                'Percepción de funcionalidad : Nada -     3': v8,
                'Percepción de funcionalidad :    4 -     7': v9,
                'Percepción de funcionalidad :    8 - Mucho': v10}
        
        list_date = res_merged['Fecha'] #['2022-01-01', '2022-02-01', '2022-03-01']
        
        res = mainkst(init_val, para, list_date, clima_tipo)
        kst_df = res.copy()
        
        print('Modelo de capital social ejecutado.')
        cond_ini_kst = True
        
        display(HTML(res.to_html()))
        
        for name_col in res.columns[res.columns != 'Fecha']:
            res_merged[name_col] = res[name_col]
        
        print('Modelo de capital social ejecutado.')
        print('Copy to clipboard.')
        @interact_manual()
        def copy():
            res.to_clipboard(index=False)
    else:
        print('Por favor, selecione la trayectoria.')
        
print('')
print('Parámetros del capital social: ')
print('')

w = interact_manual(capSocial,
                    v1 = input_as_decimal('Tasa de vinculación comunitaria', 0.4),
                    v2 = input_as_decimal('Tasa de grupo de apoyo ante eventos', 0.4),
                    v3 = input_as_decimal('Tasa de percepción positiva de ayudas del gobierno', 0.4),
                    v4 = input_as_decimal('Tasa de vinculación productiva', 0.4),
                    v5 = input_as_decimal('Tasa de no vinculación con organizaciones relacionadas a eventos externos', 0.4),
                    v6 = input_as_decimal('Tasa de vinculación familiar', 0.4),
                    v7 = input_as_decimal('Union de la comunidad ante eventos', 0.4),
                    v8 = input_as_decimal('Percepción de funcionalidad : Nada -     3', 0.4),
                    v9 = input_as_decimal('Percepción de funcionalidad :    4 -     7', 0.4),
                    v10= input_as_decimal('Percepción de funcionalidad :    8 - Mucho', 0.4))


Parámetros del capital social: 



interactive(children=(FloatText(value=0.4, description='Tasa de vinculación comunitaria', layout=Layout(width=…

<p><b>Capital Humano</b></p>

![kht](Capitales/kht.png)

In [31]:
def capHumano(v1, v2, v3, v4, v5, v6, v7, v8, v9, v10,
              v11, v12, v13, v14, v15, v16, v17):
    if cond_ini_merge == True:
        
        global cond_ini_kht, kht_df
        
        init_val = 0
        list_date = res_merged['Fecha'] # ['2022-01-01', '2022-02-01', '2022-03-01']
        
        para = {'Hogares' : v1,
                'Número de hogares que migraron debido a la inundacion' : v2,
                'Número de hogares que migraron debido a la sequia' : v3,
                'Porcentaje estado de salud - [0-100]': v4,
                'Actividad desarrollada - agricultura': v5,
                'Actividad desarrollada - ganaderia': v7,
                'Actividad desarrollada - pesca': v7,
                'Actividad desarrollada - forestal': v8,
                'Actividad desarrollada - comercio': v9,
                'Actividad desarrollada - actividad de campo': v10,
                'Actividad desarrollada - actividad no agropecuaria': v11,
                'Actividad desarrollada - Combinación': v12,
                'Nivel de educación': v13,
                'Rango de edades':v14,
                'Exp. en inundaciones (%): Nada - 3    ' : v15,
                'Exp. en inundaciones (%): 4    - 8    ' : v16,
                'Exp. en inundaciones (%): 8    - Mucho' : v17}
        
        res = mainkht(init_val, para, list_date, clima_tipo)
        kht_df = res.copy()
        
        for name_col in res.columns[res.columns != 'Fecha']:
            res_merged[name_col] = res[name_col]
        
        cond_ini_kht = True
        
        display(HTML(res.to_html()))
        print('Modelo de capital humano ejecutado.')
        print('Copy to clipboard.')
        @interact_manual()
        def copy():
            res.to_clipboard(index=False)
        
    else:
        print('Por favor, selecione la trayectoria.')
        
print('')
print('Parámetros del capital humano: ')
print('')

w = interact_manual(capHumano,
                    v1  = input_as_decimal('Hogares', 100),
                    v2  = input_as_decimal('Número de hogares que migraron debido a la inundacion', 20),
                    v3  = input_as_decimal('Número de hogares que migraron debido a la sequia', 30),
                    v4  = input_as_decimal('Porcentaje estado de salud - [0-100]', 60),
                    v5  = input_as_decimal('Actividad desarrollada - agricultura', 2),
                    v6  = input_as_decimal('Actividad desarrollada - ganaderia', 5),
                    v7  = input_as_decimal('Actividad desarrollada - pesca', 7),
                    v8  = input_as_decimal('Actividad desarrollada - forestal', 12),
                    v9  = input_as_decimal('Actividad desarrollada - comercio', 1),
                    v10 = input_as_decimal('Actividad desarrollada - actividad de campo', 3),
                    v11 = input_as_decimal('Actividad desarrollada - actividad no agropecuaria', 7),
                    v12 = input_as_decimal('Actividad desarrollada - Combinación', 5),
                    v13 = input_as_list('Nivel de educación', 'Básica Primaria', ["Ninguno", "Preescolar",
                                                                                  "Básica Primaria", "Básica secundaria",
                                                                                  "Educación media", "Tecnológica",
                                                                                  "Universitaria"]),
                    v14 = input_as_list('Rango de edades', "Entre 19 y 60 años", ["Menores de 6 años",
                                                                                  'Entre 6 y 18 años',
                                                                                  "Entre 19 y 60 años",
                                                                                  "Básica secundaria",
                                                                                  "Mayores a 60 años"]),
                    v15 = input_as_decimal('Exp. en inundaciones (%): Nada - 3    ', 14.04),
                    v16 = input_as_decimal('Exp. en inundaciones (%): 4    - 8    ', 47.85),
                    v17 = input_as_decimal('Exp. en inundaciones (%): 8    - Mucho', 38.1))


Parámetros del capital humano: 



interactive(children=(FloatText(value=100.0, description='Hogares', layout=Layout(width='50%'), style=Descript…

<p>Los capitales se calcula a partir de las relaciones mostradas en los siguientes diagrámas de Forrester.</p>

<p><b>Capital Físico</b></p>

![kfit](Capitales/kfit.png)

In [32]:
def capFisico(v1, v2, v3, v4, v5, v6, v7, v8, v9, v10,
              v11, v12, v13, v14, v15):
    if cond_ini_merge == True and cond_ini_kst == True and cond_ini_kht == True:
        
        init_val = 0
#         para = {'Vivienda - Propia' : 0.4,
#                 'Vivienda - Arriendo' : 0.4,
#                 'Vivienda - Aparceria' : 0.4,
#                 'Vivienda - Usufructo' : 0.4,
#                 'Vivienda - Comodato' : 0.4,
#                 'Vivienda - Ocupación de hecho' : 0.4,
#                 'Vivienda - Propiedad colectiva' : 0.4,
#                 'Vivienda - Adjudicatario' : 0.4,
#                 'Porcentaje de no obtención - Automovil o moto' : 0.4,
#                 'Porcentaje de no obtención - Tractor' : 0.4,
#                 'Porcentaje de no obtención - Inta. especiales' : 0.4,
#                 'Porcentaje de no obtención - Herr. agrícolas' : 0.4,
#                 'Porcentaje de SI actuación miembros del hogar' : 0.4,
#                 'Promedio de rangos de uso de la tierra' : 0.4,
#                 'Rango capital físico : Infraestructura disponible' : 0.2}

        para = {'Vivienda - Propia' : v1,
                'Vivienda - Arriendo' : v2,
                'Vivienda - Aparceria' : v3,
                'Vivienda - Usufructo' : v4,
                'Vivienda - Comodato' : v5,
                'Vivienda - Ocupación de hecho' : v6,
                'Vivienda - Propiedad colectiva' : v7,
                'Vivienda - Adjudicatario' : v8,
                'Porcentaje de no obtención - Automovil o moto' : v9,
                'Porcentaje de no obtención - Tractor' : v10,
                'Porcentaje de no obtención - Inta. especiales' : v11,
                'Porcentaje de no obtención - Herr. agrícolas' : v12,
                'Porcentaje de SI actuación miembros del hogar' : v13,
                'Promedio de rangos de uso de la tierra' : v14,
                'Rango capital físico : Infraestructura disponible' : v15}
        
        n_paso_tmp = len(kst_df)
        
        kht_df_tmp = pd.DataFrame()
        kht_df_tmp['Experiencia manejo de eventos'] = kht_df['Habilidad Manejo de inundación'].copy()
        #kht_df = pd.DataFrame(data={'Experiencia manejo de eventos': [0.5] * n_paso_tmp})
        #print('Falta la aclaración para el cálculo de la experiencia manerjo de eventos')

        res = mainkfitt(init_val, para,
                        kst_df, kht_df_tmp)
        cond_ini_kfitt = True
        
        for name_col in res.columns[res.columns != 'Fecha']:
            res_merged[name_col] = res[name_col]
        
        display(HTML(res.to_html()))
        print('Modelo de capital físico ejecutado.')
        print('Copy to clipboard.')
        @interact_manual()
        def copy():
            res.to_clipboard(index=False)
    else:
        print('Se requiere ejecutar los modelos de capital humano y capital social')
        
print('')
print('Parámetros del capital físico: ')
print('')

w = interact_manual(capFisico,
                    v1  = input_as_decimal('Vivienda - Propia', 0.4),
                    v2  = input_as_decimal('Vivienda - Arriendo', 0.4),
                    v3  = input_as_decimal('Vivienda - Aparceria', 0.4),
                    v4  = input_as_decimal('Vivienda - Usufructo', 0.4),
                    v5  = input_as_decimal('Vivienda - Comodato', 0.4),
                    v6  = input_as_decimal('Vivienda - Ocupación de hecho' , 0.4),
                    v7  = input_as_decimal('Vivienda - Propiedad colectiva' , 0.4),
                    v8  = input_as_decimal('Vivienda - Adjudicatario', 0.4),
                    v9  = input_as_decimal('Porcentaje de no obtención - Automovil o moto', 0.4),
                    v10 = input_as_decimal('Porcentaje de no obtención - Tractor', 0.4),
                    v11 = input_as_decimal('Porcentaje de no obtención - Inta. especiales', 0.4),
                    v12 = input_as_decimal('Porcentaje de no obtención - Herr. agrícolas', 0.4),
                    v13 = input_as_decimal('Porcentaje de SI actuación miembros del hogar', 0.4),
                    v14 = input_as_decimal('Promedio de rangos de uso de la tierra', 0.4),
                    v15 = input_as_decimal('Rango capital físico : Infraestructura disponible', 0.4))


Parámetros del capital físico: 



interactive(children=(FloatText(value=0.4, description='Vivienda - Propia', layout=Layout(width='50%'), style=…

<p><b>Capital Finaiciero</b></p>

![kft](Capitales/kft.png)

In [33]:
def capFinanciero(v1, v2, v3, v4, v5, v6, v7, v8, v9, v10,
                  v11, v12, v13, v14, v15):
    if cond_ini_merge == True:
        
        global cond_ini_kft
        
#         para = {'Agricultura, especies menores, pesca' : 87251,
#                 'Agricultura y especies menores' : 116207,
#                 'Agricultura, especies menores, Actividades no agropecuarias' : 93367,

#                 'Agricultura, especies menores, pesca - 1' : 120588,
#                 'Agricultura, especies menores, Actividades no agropecuarias 1' : 207143,
#                 'Agricultura y especies menores - 1' : 81531,

#                 'Agricultura, especies menores, pesca - 2' : 185516,
#                 'Agricultura, especies menores, Actividades no agropecuarias 2' : 256650,
#                 'Agricultura y especies menores - 2' : 143353,

#                 'Agricultura, especies menores, pesca - 3' : 4064694,
#                 'Agricultura, especies menores, Actividades no agropecuarias 3' : 6826230,
#                 'Agricultura y especies menores - 3' : 5839102,

#                 'Salario mínimo legal vigente - 2020' : 877802
#                 }

        para = {'Agricultura, especies menores, pesca' : v1,
                'Agricultura y especies menores' : v2,
                'Agricultura, especies menores, Actividades no agropecuarias' : v3,

                'Agricultura, especies menores, pesca - 1' : v4,
                'Agricultura, especies menores, Actividades no agropecuarias 1' : v5,
                'Agricultura y especies menores - 1' : v6,

                'Agricultura, especies menores, pesca - 2' : v7,
                'Agricultura, especies menores, Actividades no agropecuarias 2' : v8,
                'Agricultura y especies menores - 2' : v9,

                'Agricultura, especies menores, pesca - 3' : v10,
                'Agricultura, especies menores, Actividades no agropecuarias 3' : v11,
                'Agricultura y especies menores - 3' : v12,

                'Salario mínimo legal vigente - 2020' : v13
                }

        
        agric_kf_df = pd.DataFrame(data={'Fecha': res_merged['Fecha'],
#                                          'Costos agricolas' : [10000] * len(res_merged),
#                                          'Ventas agrícolas' : [12000000] * len(res_merged)})
                                         'Costos agricolas' : [v14] * len(res_merged),
                                         'Ventas agrícolas' : [v15] * len(res_merged)})
        
        print('*Costos y ventas agrícolas se suponen constantes.')
        
        res = mainkft(0, para, 
                      res_merged, res_merged,
                      res_merged, agric_kf_df)
        
        cond_ini_kft = True
        
        for name_col in res.columns[res.columns != 'Fecha']:
            res_merged[name_col] = res[name_col]
        
        display(HTML(res.to_html()))
        print('Modelo de capital financiero ejecutado.')
        print('Copy to clipboard.')
        @interact_manual()
        def copy():
            res.to_clipboard(index=False)
        
    else:
        print('Se requiere obtener los modelos en conjunto')
        
print('')
print('Parámetros del capital financiero : ')
print('')

w = interact_manual(capFinanciero,
                    v1  = input_as_decimal('Agricultura, especies menores, pesca', 87251), 
                    v2  = input_as_decimal('Agricultura y especies menores', 116207),
                    v3  = input_as_decimal('Agricultura, especies menores, Actividades no agropecuarias', 93367),
                    
                    v4  = input_as_decimal('Agricultura, especies menores, pesca - 1', 120588),
                    v5  = input_as_decimal('Agricultura, especies menores, Actividades no agropecuarias 1', 207143),
                    v6  = input_as_decimal('Agricultura y especies menores - 1' , 81531),
                    
                    v7  = input_as_decimal('Agricultura, especies menores, pesca - 2', 185516),
                    v8  = input_as_decimal('Agricultura, especies menores, Actividades no agropecuarias 2', 256650),
                    v9  = input_as_decimal('Agricultura y especies menores - 2' , 143353),
                    
                    v10 = input_as_decimal('Agricultura, especies menores, pesca - 3', 4064694),
                    v11 = input_as_decimal('Agricultura, especies menores, Actividades no agropecuarias 3', 6826230),
                    v12 = input_as_decimal('Agricultura y especies menores - 3' , 5839102),
                    
                    v13 = input_as_decimal('Salario mínimo legal vigente - 2020', 877802),
                    
                    v14 = input_as_decimal('Costos agricolas', 10000),
                    v15 = input_as_decimal('Ventas agrícolas', 12000000))


Parámetros del capital financiero : 



interactive(children=(FloatText(value=87251.0, description='Agricultura, especies menores, pesca', layout=Layo…

Dibujar las gráficas de todas las variables modeladas

In [39]:
@interact_manual()
def draw_results():
    
    for name_col in res_merged_fixed.columns[res_merged_fixed.columns != 'Fecha']:
        
        fig, ax = plt.subplots(1, 1, figsize=(15, 10) )
        ax.plot(res_merged_fixed['Fecha'], res_merged_fixed[name_col])
        ax.set_title(name_col)
        ax.grid()
        plt.show()

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…